In [1]:
include("PPO_parallel.jl")

PPO (generic function with 1 method)

In [2]:
env = init_Env(["14", "10", "64", "3", "100", "100", "0.01", "2", "100", "4", "10.0", "0.4", "1.0", "0.7", "0.5"])

max_turn:  14
num_player:  10
middle_dim:  64
depth:  3
training_step:  100
batch_size:  100
entropy weight:  0.01
num_simulation:  2
system parm: Ω=10.0, ξ=0.4, Jz=1.0, Jx=0.7, hz=0.5, tstep=100


Env(14, 10, 2, 3, 1, 6, 84, 64, 7, 3, 100, 200, 100, 0.98f0, 0.94f0, 0.2f0, 0.01f0, 0.002f0, 2, 100, 4, 10.0f0, 0.4f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.4f0 + 0.0f0im -0.4f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.4f0 - 0.0f0im -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.0062831854f0)

In [3]:
max_hist = [-12.0f0]
model_0 = Chain(Dense(zeros(Float32, env.output, env.input_dim))) |> gpu
model = Chain(Dense(env.input_dim, env.middle_dim), BatchNorm(env.middle_dim), Tuple(Chain(Parallel(+, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, env.middle_dim, relu),Dense(env.middle_dim, env.middle_dim, relu)), identity)) for i in 1:env.depth)..., Flux.flatten, Flux.Parallel(vcat, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), env.act_ind, tanh2)), Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), 1, tanh10)))) |> gpu
    opt = Flux.Optimiser(WeightDecay(1f-6), Adam(env.η))

Flux.Optimise.Optimiser(Any[WeightDecay(1.0f-6), Adam(0.0020000000949949026, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())])

In [10]:
old_model = deepcopy(model)

Chain(
  Dense(84 => 64),                      # 5_440 parameters
  BatchNorm(64),                        # 128 parameters, plus 128
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Flux.flatten,
  Parallel(
    vcat,
    Chain(
      BatchNorm(6

In [20]:
Y = model(cu(make_image(env, Int[])))

7×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

: 

In [5]:
replay_buffer = init_buffer(1000, env.batch_size)

ReplayBuffer(Agent[], 1000, 100, Dict{Vector{Int64}, Float32}())

In [14]:
run_selfplay!(env, replay_buffer, model, max_hist)

In [15]:
@show max_hist

max_hist = Float32[-12.0, 4.8353996, 4.8353996, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 6.001385, 8.901027, 8.901027]


20-element Vector{Float32}:
 -12.0
   4.8353996
   4.8353996
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   6.001385
   8.901027
   8.901027

In [6]:
function AlphaZero_ForPhysics_check(env::Env)
    ld = []
    max_hist::Vector{Float32} = [-12.0f0]

    #model_0 = Chain(Dense(zeros(Float32, env.output, env.input_dim))) |> gpu
    #model = Chain(Dense(env.input_dim, env.middle_dim), BatchNorm(env.middle_dim), Tuple(Chain(Parallel(+, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, env.middle_dim, relu),Dense(env.middle_dim, env.middle_dim, relu)), identity)) for i in 1:env.depth)..., Flux.flatten, Flux.Parallel(vcat, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), env.act_ind, tanh2)), Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), 1, tanh10)))) |> gpu
    #model = Chain(Dense(env.input_dim, env.middle_dim), Tuple(Chain(Parallel(+, Chain(Dense(env.middle_dim, env.middle_dim, relu),Dense(env.middle_dim, env.middle_dim, relu)), identity)) for i in 1:env.depth)..., Flux.flatten, Flux.Parallel(vcat, Chain(Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), env.act_ind, tanh2)), Chain(Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), 1, tanh10)))) |> gpu
    model = Chain(Dense(env.input_dim, env.middle_dim), BatchNorm(env.middle_dim), Tuple(Chain(Parallel(+, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, env.middle_dim, relu),Dense(env.middle_dim, env.middle_dim, relu)), identity)) for i in 1:env.depth)..., Flux.flatten, Flux.Parallel(vcat, Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), env.act_ind, tanh2)), Chain(BatchNorm(env.middle_dim), Dense(env.middle_dim, div(env.middle_dim,2), relu), Tuple(Dense(div(env.middle_dim,2), div(env.middle_dim,2), relu) for i in 1:3)..., Dense(div(env.middle_dim,2), 1, tanh10)))) |> gpu
    opt = Flux.Optimiser(WeightDecay(1f-6), Adam(env.η))

    replay_buffer = init_buffer(1000, env.batch_size)
    #old_model = deepcopy(model) |> gpu

    #for it in 1:env.num_simulation
    for it in 1:1
        println("=============")
        println("it=$(it);")
        old_model = deepcopy(model) |> gpu
        if(it%(env.num_simulation/10)==0)
            println("=============")
            println("it=$(it);")
            println("max score: $(max_hist[end])")
            #k = [keys(replay_buffer.scores)...]
            #inds = findall(s -> replay_buffer.scores[s] == max_hist[end], k)
            #for i in inds
            #    println("$(k[i])")
            #end
        end
        
        if(it == 1)
            run_selfplay!(env, replay_buffer, model, max_hist)
        else
            run_selfplay!(env, replay_buffer, model, max_hist)
        end

        for xx in 1:10
            ll = train_model!(env, replay_buffer, model, old_model, opt)
            println("loss: $(ll)")
        end
        println(model(cu(make_image(env, Int[]))))
        println(model(cu(make_image(env, Int[6]))))
        #push!(ld,ll)
        
    end
    
    return ld, max_hist, model, replay_buffer.scores
end

AlphaZero_ForPhysics_check (generic function with 1 method)

In [18]:
@everywhere function loss(env::Env, images_batch,legals_batch, actions_batch, rewards_batch, model::Chain, old_model::Chain)
    L::Float32 = 0.0f0
    
    for b in 1:env.batch_size
        val_next = 0.0f0
        adv = 0.0f0
        reward = rewards_batch[b]
        #hist = copy(game.history)
        for it in 1:length(actions_batch[b])
            image = images_batch[b][it] |> gpu
            y = model(image) |> cpu
            y_old = old_model(image) |> cpu

            policy_log = y[1:end-1, 1]
            val = y[end, 1]
            #policy_log_old = y_old[1:end-1, 1]
            #policy = softmax([policy_log[i] for i in legals_batch[b][it]])
            #policy_old = softmax([policy_log_old[i] for i in legal])

            delta = (reward + env.γ * val_next - val)
            #/10.0f0

            #adv = delta + env.γ * env.λ * adv
            #a = actions_batch[b][it]
            #rt = exp(step(it)*(policy_log[a,1]-policy_log_old[a,1]))
            
            L += delta*delta
            #min(rt*adv, clip(rt, env.ϵ)*adv) + delta^2
            #  - step(it) * env.E * (policy' * log.(policy))
            #L += delta^2  - step(it)*env.E * (policy' * log.(policy))

            val_next = val
            reward = 0.0f0
        end
    end
    return L / env.batch_size
    # + env.C * sum(sqnorm, Flux.params(model))
end
@everywhere function sample_batch(env::Env, buffer::ReplayBuffer)
    games = sample(buffer.buffer, env.batch_size, replace=true)
    images_batch::Vector{Vector{Matrix{Int}}} = []
    legals_batch::Vector{Vector{Vector{Int}}} = []
    actions_batch::Vector{Vector{Int}} = []
    rewards_batch::Vector{Float32} = []
    for game in games
        l = length(game.history)
        image_batch::Vector{Matrix{Int}} = []
        legal_batch::Vector{Vector{Int}} = []
        branch_left::Vector{Int} = [-1]
        for turn in 1:l
            push!(image_batch, make_image(env, game.history, turn))
            legal = legal_action(env, game.history[1:turn], branch_left)
            push!(legal_batch, legal)
            if(game.history[turn] <= env.val_num)
                pop!(branch_left)
            elseif(game.history[turn] <= env.val_num+env.br_num)
                push!(branch_left, game.history[turn])
            end
        end
        push!(images_batch, reverse(image_batch))
        push!(legals_batch, reverse(legal_batch))
        push!(actions_batch, reverse(game.history))
        push!(rewards_batch, game.reward)
    end
    return images_batch, legals_batch, actions_batch, rewards_batch
end
#gpu並列化予定
@everywhere function train_model!(env::Env, buffer::ReplayBuffer, model::Chain, old_model::Chain, opt)

    
    images_batch, legals_batch, actions_batch, rewards_batch = sample_batch(env, buffer)
    val, grads = Flux.withgradient(Flux.params(model)) do
        loss(env, images_batch, legals_batch, actions_batch, rewards_batch, model, old_model)
    end
    Flux.Optimise.update!(opt, Flux.params(model), grads)
    return val
end

In [19]:
ld0, max_hist0, model0, scores0 = AlphaZero_ForPhysics_check(env)

it=1;


LoadError: GPU compilation of MethodInstance for (::GPUArrays.var"#broadcast_kernel#38")(::CUDA.CuKernelContext, ::CuDeviceMatrix{Float32, 1}, ::Base.Broadcast.Broadcasted{CUDA.CuArrayStyle{2}, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}, Zygote.var"#1396#1400"{Int64}, Tuple{Base.Broadcast.Extruded{Matrix{Float32}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}, Base.Broadcast.Extruded{CuDeviceMatrix{ForwardDiff.Dual{Nothing, Float32, 1}, 1}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}}}, ::Int64) failed
KernelError: passing and using non-bitstype argument

Argument 4 to your kernel function is of type Base.Broadcast.Broadcasted{CUDA.CuArrayStyle{2}, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}}, Zygote.var"#1396#1400"{Int64}, Tuple{Base.Broadcast.Extruded{Matrix{Float32}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}, Base.Broadcast.Extruded{CuDeviceMatrix{ForwardDiff.Dual{Nothing, Float32, 1}, 1}, Tuple{Bool, Bool}, Tuple{Int64, Int64}}}}, which is not isbits:
  .args is of type Tuple{Base.Broadcast.Ex

In [36]:
modelc = model0 |> cpu

Chain(
  Dense(84 => 64),                      # 5_440 parameters
  BatchNorm(64),                        # 128 parameters, plus 128
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      identity,
    ),
  ),
  Flux.flatten,
  Parallel(
    vcat,
    Chain(
      BatchNorm(6

In [39]:
modelc(zeros(Float32, env.input_dim, 1))

7×1 Matrix{Float32}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [38]:
model0(cu(zeros(Float32, env.input_dim, 1)))

7×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN